In [1]:
# import
import aiohttp
import pkuseg
import pymongo
import zhconv
from aip import AipNlp
from motor import motor_asyncio
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
    
# CONFIG
if platform.system() == 'Windows':
    # CODE_PATH
    CODE_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/code')
    DATA_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/data')
    %cd {CODE_PATH}
elif platform.system() == 'Linux':
    # CODE_PATH
    CODE_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/code'
    DATA_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/data'
    %cd {CODE_PATH}

# Baidu NLP
def get_token(accounts):
    '''get API tokens'''
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials"
    for account in accounts:
        # client_id 为官网获取的AK， client_secret 为官网获取的SK
        param = {'client_id': account['apiKey'],
                 'client_secret': account['secretKey']}
        resp = requests.get(url, param).json()
        yield resp['access_token']
        
accounts = [{'appId': '16105374', # ross.zhu@outlook.com
            'apiKey': 'zShobECEYX2B2CNrcz3QsM3I',
            'secretKey': 'xM8g1lkhTd8b7DYGv2H9j7g2hvthPGE5'},
           {'appId': '16218655', # 18868819096
            'apiKey': 'AG2ygGZbCPqDW25ZC6FinQRy',
            'secretKey': 'pcx8pFwjVhD0oGDsdm58bzt7Zm9zFpWk'},
           {'appId': '16226200', # 13566633705
            'apiKey': 'ymmGiPQoAoSF2KhcGVKNZEVj',
            'secretKey': '78oQXsRtdmRcINEGtP3aP2y4sKA7HerU'},
           {'appId': '16226243', # 13606586786
            'apiKey': 'jIy3T25rYR1Kj9BlCKPcHbYe',
            'secretKey': 'jORNEeu7Qnl6L1Hr07NomigmsUtXlloL'},
           {'appId': '16230746', # 李锐
            'apiKey': 't6sWsoIYinoGBgwbLiCopQHR',
            'secretKey': 'evq4UygxQ555LXzVzaDWQS2TfGNUHrYf'},
           {'appId': '16307961', # 俞嘉炜
            'apiKey': 'TKIEhGe1fIY89VlYm8yjuDZW',
            'secretKey': '6kDBxZnGZEsK4y9ntg94VOBSIjLS0UQs'},
           {'appId': '16305932', # 芳菲
            'apiKey': 'Y7NjBe2sqhfIjSIaB51wvlHH',
            'secretKey': 'lqLW2XeSj0xBvCbUASh0E2wuMmW6e4ox'},
           {'appId': '16305931', # 小毫生
            'apiKey': 'D2GGEWS80KsU9mtft1WnuR0N',
            'secretKey': '8FbvLeGD5aXLL6bpKSEZoG7CRt1Q9oqo'},
           {'appId': '16306399', # 忻小宇
            'apiKey': 'E89qU1OMrwvyanH7xixkFVrR',
            'secretKey': 'ljPLgCtzOKLZuG56DPX9hvrVFHRsWHqf'},
           {'appId': '16306475', # 杨老板
            'apiKey': 'yehP04KXoWADTGUyLF8WK0hN',
            'secretKey': 'jfp3wxqupbWfEoNi7L316LdfbUmDmRUo'},
           ]
tokens = list(get_token(accounts))
baidu_clients = [AipNlp(**account) for account in accounts] 

# MongoDB-pymongo
pymongo_client = MongoClient('localhost', 27018)
db = pymongo_client.exchange
coll_embedding_bd = db.embedding_bd
coll_vocab_without_emb = db.vocab_without_emb
coll_vocab_to_request = db.vocab_to_request
coll_vocab_pos = db.vocab_pos
coll_vocab_ne = db.vocab_ne
coll_text_seg_bd = db.text_seg_bd
coll_title_seg_bd = db.title_seg_bd
coll_test = db.test

C:\Users\rossz\OneDrive\STN\10-FirstYear\code


In [2]:
%%time
def build_requests(cmt_path, cmt_coll):
    cmt_text = pickle.load(open(cmt_path, 'rb'))
    cmt_text = [(i, x) for i, x in enumerate(cmt_text)]

    segged_cmt_id = set(x['id'] for x in cmt_coll.find(projection={'id':1, '_id':0}))
    print(f'{len(segged_cmt_id)} docs have been segged.')
    
    request_cmt_id = set(range(len(cmt_text))) - segged_cmt_id
    request_cmt = [(i, doc) for i, doc in cmt_text if i in request_cmt_id]
    print(f'{len(request_cmt_id)} docs to reqeust.')
    
    vocab_to_request = set(x['word'] for x in coll_vocab_to_request.find(projection={'word':1, '_id':0}))
    print(f'{len(vocab_to_request)} words to request')
    
    return request_cmt, vocab_to_request

# request_cmt_title, request_cmt_id = build_requests('../data/cmt_title_clean.pkl', coll_title_seg_bd)
request_cmt_text, vocab_to_request = build_requests('../data/cmt_text_clean.pkl', coll_text_seg_bd)

6894896 docs have been segged.
37510263 docs to reqeust.
6748490 words to request
Wall time: 2min 6s


In [ ]:
async def get_lex(text, token):
    base_url = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer'
    url_params = {'access_token': token,
                  'charset': 'UTF-8'}
    headers = {'Content-Type': 'application/json'}
    request_text = {'text': text}

    async with aiohttp.ClientSession() as session:
        async with session.request('POST', url=base_url, json=request_text, headers=headers, params=url_params) as resp:
            response = await resp.json()
            return response
        
async def seg_one(idx, text, token, coll_cmt, bucket, download_delay, len_tasks):
    global t0, count_all, last_count_all, count_success, count_empty_input,\
        count_input_too_long, count_limit_reached,\
        count_other_error, count_duplicate, count_mongo_error,\
        vocab_to_request
    try:
        
        # if empty text, directly write to db
        # else, send request and get response
        if text == '':
            count_empty_input += 1
            coll_cmt.insert_one({'id': idx, 'text_seg': '', 'text_seg2': ''})
        else:
            await asyncio.sleep(download_delay)
            # if longer than 20000 bytes, truncate
            is_too_long = False
            if len(text.encode('utf-8')) >= 20000:
                text = text.encode('utf-8')[:20000].decode('utf-8', errors='ignore')
                is_too_long = True
            async with bucket:
                    response = await get_lex(text, token)

        # process errors in response
        # if no error, generate input doc
        if 'error_code' in response:
            if response['error_code'] == 18: # limit reached
                count_limit_reached += 1
            elif response['error_code'] == 282131: # input too long
                count_input_too_long += 1
            else:
                print(f'other baidu error: request:"{text[:20]}...", response:"{response}"')
                print(token)
                count_other_error += 1
        else:
            # process successful response
            response = response['items']
            vocab_coarse = [item['item'] for item in response]
            vocab_granular = list(itertools.chain.from_iterable([item['basic_words'] for item in response]))
            text_seg = ' '.join(vocab_coarse)
            text_seg2 = ' '.join(vocab_granular)
            pos = {item['item']: item['pos'] for item in response if item['pos'] != ''}
            ne = {item['item']: item['ne'] for item in response if item['ne'] != ''}

            # write coll_text_seg_bd; coll_vocab_to_request
            with capture_output():
                coll_cmt.insert_one({'id': idx, 'text_seg': text_seg, 'text_seg2': text_seg2, 'is_too_long': is_too_long})
                count_success += 1
                for word in itertools.chain(vocab_coarse, vocab_granular):
                    if word not in vocab_to_request:
                        coll_vocab_to_request.insert_one({'word': word})
                
    except DuplicateKeyError:
        count_duplicate += 1
    except:
        print(f'Unknow Exception: {sys.exc_info()[:-1]}')
        count_other_error += 1
    finally:
    # log
        if count_all % 10000 == 0:
            print(f'{count_all}/{len_tasks}, {Now()} ({time.time()-t0:.1f}s / {(count_all-last_count_all)/(time.time()-t0):.1f}QPS)')
            print(f'success:{count_success}, limit-reached:{count_limit_reached}, empty-input-in-success:{count_empty_input}, dup:{count_duplicate}, input-too-long:{count_input_too_long}, other-baidu-error:{count_other_error}, other-mongo-error:{count_mongo_error}')
            print('---------------------')
            count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0
            t0 = time.time()
            last_count_all = count_all
    
    
async def download_many(tokens, request_cmt, coll_cmt, max_concur_req, download_delay):
    global count_all
    tokens_iter = itertools.cycle(tokens)
    len_tasks = len(request_cmt)
    # restrict concurrenty
    bucket = AsyncLeakyBucket(max_concur_req, 1)
    
    # generate tasks
    print(f'generating tasks...{Now()}')
    tasks = []
    for idx, text in request_cmt:
        task = seg_one(idx, text, next(tokens_iter), coll_cmt, bucket, download_delay, len_tasks)
        tasks.append(task)
    print(f'tasks generated {Now()}')
        
    # schedule tasks
    for future in tqdm(asyncio.as_completed(tasks), total=len_tasks):
        count_all += 1
        await future

# run the main
t0 = time.time()
count_all, last_count_all, count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0, 0

asyncio.run(download_many(tokens, request_cmt_text[:5000000], coll_text_seg_bd, max_concur_req=31, download_delay=0))

  0%|▎                                                                   | 20908/5000000 [13:29<44:35:55, 31.01it/s]

Unknow Exception: (<class 'aiohttp.client_exceptions.ContentTypeError'>, ContentTypeError("0, message='Attempt to decode JSON with unexpected mimetype: text/html;charset=utf-8'"))


  1%|▍                                                                   | 28505/5000000 [17:30<43:33:59, 31.70it/s]